In [138]:
from pymongo import MongoClient
import pandas as pd
import json
import geojson
import os
import requests
from dotenv import load_dotenv
import time
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from folium import Icon

## VISUALIZING ##

In [74]:
client = MongoClient("localhost:27017")
db = client["cities_project"]

**IMPORTING JSON INTO DF**

In [75]:
def mongodb_to_df(name_collection):

    '''Function to get the collections from mongo and turn them to DF
    It takes 1 arg:
    - collection name.
    Also checks for duplicate columns and drops them, only based on lat and long. 
    '''

    df = pd.DataFrame(list(db[name_collection].find()))
    df_drop_duplicates = df.drop_duplicates(subset=df.columns.difference(["name"]))

    return df_drop_duplicates

    SAN FRANCISCO

In [76]:
SFstarbucks = mongodb_to_df("SFstarbucks")
SFbaskStadium = mongodb_to_df("SFbaskStadium")
SFschools = mongodb_to_df("SFschools")
SFairport = mongodb_to_df("SFairport")
SFtrainStations = mongodb_to_df("SFtrainStations")
SFpet_grooming = mongodb_to_df("SFpet_grooming")
SFdog_parks = mongodb_to_df("SFdog_parks")
SFbars_clubs = mongodb_to_df("SFbars_clubs")
SFvegan_vegetarian = mongodb_to_df("SFvegan_vegetarian")
SFvegan_vegetarian.head()

,_id,name,lat,lon
0,6547ac546d42dc580d4d6464,Ferry Building Marketplace,37.795536,-122.393425
1,6547ac546d42dc580d4d6465,Mission Dolores Park,37.759655,-122.426912
2,6547ac546d42dc580d4d6466,Berkeley Bowl,37.853335,-122.290239
3,6547ac546d42dc580d4d6467,Whole Foods,37.764381,-122.402680
4,6547ac546d42dc580d4d6468,Whole Foods,37.789980,-122.423402


In [77]:
'''def drop_rows_notStarb(dataframe):

    Function to make sure that all the caffes are starbucks. Because looking at the dataframe, that wan't happenning.
    Takes 1 arg:
    - Dataframe

    starbucks = dataframe['name'] == 'Starbucks'
    
    filtered_dataframe = dataframe[starbucks]
    
    return filtered_dataframe'''

"def drop_rows_notStarb(dataframe):\n\n    Function to make sure that all the caffes are starbucks. Because looking at the dataframe, that wan't happenning.\n    Takes 1 arg:\n    - Dataframe\n\n    starbucks = dataframe['name'] == 'Starbucks'\n    \n    filtered_dataframe = dataframe[starbucks]\n    \n    return filtered_dataframe"

In [78]:
'''SFstarbucks = drop_rows_notStarb(SFstarbucks)
SFstarbucks'''

'SFstarbucks = drop_rows_notStarb(SFstarbucks)\nSFstarbucks'

    NEW YORK

In [130]:
NYCstarbucks = mongodb_to_df("NYCstarbucks")
NYCbaskStadium = mongodb_to_df("NYCbaskStadium")
NYCschools = mongodb_to_df("NYCschools")
NYCairport = mongodb_to_df("NYCairport")
NYCtrainStations = mongodb_to_df("NYCtrain_stations")
NYCpet_grooming = mongodb_to_df("NYCpet_grooming")
NYCdog_parks = mongodb_to_df("NYCdog_parks")
NYCbars_clubs = mongodb_to_df("NYCbars_clubs")
NYCvegan_vegetarian = mongodb_to_df("NYCvegan_vegetarian")
NYCtrainStations.sample()

,_id,name,lat,lon
20,6547ad70a39b9f8989a37c62,Grand Central Terminal,40.752724,-73.97716


In [80]:
'''NYCstarbucks = drop_rows_notStarb(NYCstarbucks)
NYCstarbucks'''

'NYCstarbucks = drop_rows_notStarb(NYCstarbucks)\nNYCstarbucks'

    SEATTLE

In [136]:
STstarbucks = mongodb_to_df("STstarbucks")
STbaskStadium = mongodb_to_df("STbaskStadium")
STschools = mongodb_to_df("STschools")
STairport = mongodb_to_df("STairport")
STtrainStations = mongodb_to_df("STtrain_stations")
STpet_grooming = mongodb_to_df("STpet_grooming")
STdog_parks = mongodb_to_df("STdog_parks")
STbars_clubs = mongodb_to_df("STbars_clubs")
STvegan_vegetarian = mongodb_to_df("STvegan_vegetarian")


In [82]:
'''STstarbucks = drop_rows_notStarb(STstarbucks)
STstarbucks'''

'STstarbucks = drop_rows_notStarb(STstarbucks)\nSTstarbucks'

**START MAPPING**

First Import de collections from MongoDB

In [83]:
def mongodb_to_df_geo(name_collection):

    '''Function to get the collections from mongo and turn them to DF
    It takes 1 arg:
    - collection name. 
    '''
    json = db.get_collection(name_collection) 

    return json

    SAN FRANCISCO

In [107]:
SanFranciscoNeigh = mongodb_to_df_geo("SanFranciscoNeigh")

    NEW YORK

In [85]:
NewYorkNeigh = mongodb_to_df_geo("NewYorkNeigh")

    SEATTLE

In [86]:
SeattleNeigh = mongodb_to_df_geo("SeattleNeigh")

read geojson, because the collections from mongo don't work.

In [113]:
def read_geojson(relative_path):
    ''' Function to read geojson.
    It takes 1 arg:
    - Relative path of file.
    '''
    with open(relative_path) as f:
        return gpd.read_file(f)

In [116]:
SF_gj = read_geojson("../data/SanFrancisco.Neighborhoods.json")
NYC_gj = read_geojson("../data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson")
ST_gj = read_geojson("../data/Neighborhood_Map_Atlas_Neighborhoods.geojson")

Markers

LOCATIONS

In [97]:
san_francisco_loc = [37.75404,-122.44721]
new_york_loc = [40.79724,-73.94968]
seattle_loc = [47.60065,-122.31819]

Funcion para pins de cada tipo, despues llamar a estas funciones cuando se hace el mapa para tener todos los pins. 

## HEAT MAPS ##

In [123]:

def heat_map(name,loc,neigh_geojson, starbucks_df,stadium_df, schools_df, airport_df, trainstations_df, pet_grooming_df, dog_parks_df, bars_clubs_df,vegan_vegetarian_df):


    ''' Function to create heat maps with all the points of interest for each city.

    takes 12 arguments:
    - geojson of the city
    - the 9 dataframes of points of interests
    - name to save the heatmap
    - location.

    '''


    heatmap = folium.Map(location=loc, zoom_start=12)

    starbucks_coords = []
    stadium_coords = []
    school_coords = []
    trainstations_coords = []
    airport_coords = []
    pet_grooming_coords = []
    dog_parks_coords = []
    bars_clubs_coords = []
    vegan_vegetarian_coords = []


    geometry_starbucks = gpd.points_from_xy(starbucks_df['lon'], starbucks_df['lat'])
    starbucks_places = gpd.GeoDataFrame(starbucks_df, geometry=geometry_starbucks)

    for polygon in neigh_geojson.geometry:

        filtered_starbucks_places = starbucks_places[starbucks_places.geometry.within(polygon)]

        for _, row in filtered_starbucks_places.iterrows():
            starbucks_coords.append([row['lat'], row['lon']])

    geometry_stadium = gpd.points_from_xy(stadium_df['lon'], stadium_df['lat'])
    stadiums = gpd.GeoDataFrame(stadium_df, geometry=geometry_stadium)  

    for polygon in neigh_geojson.geometry:

        filtered_stadiums = stadiums[stadiums.geometry.within(polygon)]

        for _, row in filtered_stadiums.iterrows():
            stadium_coords.append([row['lat'], row['lon']])

    geometry_schools = gpd.points_from_xy(schools_df['lon'], schools_df['lat'])
    schools = gpd.GeoDataFrame(schools_df, geometry=geometry_schools)

    for polygon in neigh_geojson.geometry:

        filtered_schools = schools[schools.geometry.within(polygon)]

        for _, row in filtered_schools.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_airport = gpd.points_from_xy(airport_df['lon'], airport_df['lat'])
    airport = gpd.GeoDataFrame(airport_df, geometry=geometry_airport)

    for polygon in neigh_geojson.geometry:

        filtered_airport = airport[airport.geometry.within(polygon)]

        for _, row in filtered_airport.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_trainstations = gpd.points_from_xy(trainstations_df['lon'], trainstations_df['lat'])
    trainstations = gpd.GeoDataFrame(trainstations_df, geometry=geometry_trainstations)

    for polygon in neigh_geojson.geometry:

        filtered_trainstations = trainstations[trainstations.geometry.within(polygon)]

        for _, row in filtered_trainstations.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_pet_grooming = gpd.points_from_xy(pet_grooming_df['lon'], pet_grooming_df['lat'])
    pet_grooming = gpd.GeoDataFrame(pet_grooming_df, geometry=geometry_pet_grooming)

    for polygon in neigh_geojson.geometry:

        filtered_pet_grooming = pet_grooming[pet_grooming.geometry.within(polygon)]

        for _, row in filtered_pet_grooming.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_dog_parks = gpd.points_from_xy(dog_parks_df['lon'], dog_parks_df['lat'])
    dog_parks = gpd.GeoDataFrame(dog_parks_df, geometry=geometry_dog_parks)

    for polygon in neigh_geojson.geometry:

        filtered_dog_parks = dog_parks[dog_parks.geometry.within(polygon)]

        for _, row in filtered_dog_parks.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_bars_clubs = gpd.points_from_xy(bars_clubs_df['lon'], bars_clubs_df['lat'])
    bars_clubs = gpd.GeoDataFrame(bars_clubs_df, geometry=geometry_bars_clubs)

    for polygon in neigh_geojson.geometry:

        filtered_bars_clubs = bars_clubs[bars_clubs.geometry.within(polygon)]

        for _, row in filtered_bars_clubs.iterrows():
            school_coords.append([row['lat'], row['lon']])

    geometry_vegan_vegetarian = gpd.points_from_xy(vegan_vegetarian_df['lon'],vegan_vegetarian_df['lat'])
    vegan_vegetarian = gpd.GeoDataFrame(vegan_vegetarian_df, geometry=geometry_vegan_vegetarian)

    for polygon in neigh_geojson.geometry:

        filtered_vegan_vegetarian = vegan_vegetarian[vegan_vegetarian.geometry.within(polygon)]

        for _, row in filtered_vegan_vegetarian.iterrows():
            school_coords.append([row['lat'], row['lon']])

        HeatMap(starbucks_coords).add_to(heatmap)

        HeatMap(stadium_coords).add_to(heatmap)

        HeatMap(school_coords).add_to(heatmap)

        HeatMap(airport_coords).add_to(heatmap)

        HeatMap(trainstations_coords).add_to(heatmap)

        HeatMap(pet_grooming_coords).add_to(heatmap)

        HeatMap(dog_parks_coords).add_to(heatmap)

        HeatMap(bars_clubs_coords).add_to(heatmap)

        HeatMap(vegan_vegetarian_coords).add_to(heatmap)
    
        heatmap.save(f"../figures/heatmap{name}")

        return heatmap

    SAN FRANCISCO

In [131]:
SFheatmap = heat_map("SFheatmap",san_francisco_loc, SF_gj, SFstarbucks,SFbaskStadium, SFschools, SFairport, SFtrainStations, SFpet_grooming, SFdog_parks, SFbars_clubs,SFvegan_vegetarian)
SFheatmap

    NEW YORK CITY

In [132]:
NYCheatmap = heat_map("NYCheatmap",new_york_loc, NYC_gj, NYCstarbucks,NYCbaskStadium, NYCschools, NYCairport, NYCtrainStations, NYCpet_grooming, NYCdog_parks, NYCbars_clubs,NYCvegan_vegetarian)
NYCheatmap

    SEATTLE

In [137]:
STheatmap = heat_map("STheatmap",seattle_loc, ST_gj, STstarbucks,STbaskStadium, STschools, STairport, STtrainStations, STpet_grooming, STdog_parks, STbars_clubs,STvegan_vegetarian)
STheatmap

From what we can see in these heat maps, we can already discard Seattle, as most of the city does not have our requirements.

Although New York is has more heated points, there's also a good sum of places in San Francisco with all our requirements. 

To decide between New York and San Francisco, we can compare the airports and train stations distances.

**NEW YORK VS SAN FRANCISCO**

In [151]:
def travel(map_name, loc, geojson_data, airports_df, train_stations_df):

    ''' Function to get airports and train stations with markers in a city.
    It takes 5 arg:
    - map name
    - location
    - geo json
    - arport df
    - train df.
    '''

    map = folium.Map(location= loc, zoom_start=12)

    airports = gpd.GeoDataFrame(airports_df, geometry=gpd.points_from_xy(airports_df['lon'], airports_df['lat']))
    train_stations = gpd.GeoDataFrame(train_stations_df, geometry=gpd.points_from_xy(train_stations_df['lon'], train_stations_df['lat']))

    for polygon in geojson_data.geometry:

        airports_f = airports[airports.geometry.within(polygon)]

        for _, row in airports_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          icon=folium.Icon(icon='fa-plane', prefix='fa',color='white', icon_color='darkblue'),
                          popup=row['name']).add_to(map) 

    for polygon in geojson_data.geometry:

        train_stations_f = train_stations[train_stations.geometry.within(polygon)]

        for _, row in train_stations_f.iterrows():
            folium.Marker([row['lat'], row['lon']], 
                          tooltip=["station"], 
                          icon=folium.Icon(icon='fa-train', prefix='fa',color='white', icon_color='black'),
                          popup=row['name']).add_to(map) 

    map.save(f"../figures/heatmap{map_name}")

    return map

    NEW YORK AIRPORTS AND TRAIN STATIONS

In [152]:
NYtransport_map = travel("NYtransport_map", new_york_loc, NYC_gj, NYCairport, NYCtrainStations)
NYtransport_map

    SAN FRANCISCO AIRPORTS AND TRAIN STATIONS

In [150]:
SFtransport_map = travel("SFtransport_map", san_francisco_loc, SF_gj, SFairport, SFtrainStations)
SFtransport_map

For what we can see in those 2 maps. San Francisco doesn't have a close airport. And the way transport to get there, there are not as many possibilities.

Insted in New York, there are at least 3 closer airports, and a huge multitude of t¡ways to get there, from a lot of places in the city. 

Most of them Manhattan.

This is why, the city I'm going to choose to put my company in New York

In [ ]:
lat = 35.551389
lon = 139.743056
map_1 = folium.Map(location=[lat, lon], zoom_start=15)
club = folium.Marker(location=[35.551389, 139.743056], tooltip="Education", icon=folium.Icon(icon='fa-building', prefix='fa',color='black', icon_color='white'))
club.add_to(map_1)
for index, row in df_11.iterrows():
    lat_shop = row["lat"]
    lon_shop = row["lon"]
    club = folium.Marker(location=[lat_shop, lon_shop], tooltip="Education", icon=folium.Icon(icon='fa-graduation-cap', prefix='fa',color='blue', icon_color='white'))
    club.add_to(map_1)

In [99]:

with open("../data/SanFrancisco.Neighborhoods.json") as f:
    geojson_data = gpd.read_file(f)

starbucks_places = gpd.GeoDataFrame(SFstarbucks, geometry=gpd.points_from_xy(SFstarbucks['lon'], SFstarbucks['lat']))


SFmap = folium.Map(location=san_francisco_loc, zoom_start=12)

for polygon in geojson_data.geometry:

    filtered_starbucks_places = starbucks_places[starbucks_places.geometry.within(polygon)]

    for _, row in filtered_starbucks_places.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(SFmap)


geometry_stadium = gpd.points_from_xy(SFbaskStadium['lon'], SFbaskStadium['lat'])
stadiums = gpd.GeoDataFrame(SFbaskStadium, geometry=geometry_stadium)

for polygon in geojson_data.geometry:

    filtered_stadiums = stadiums[stadiums.geometry.within(polygon)]

    for _, row in filtered_stadiums.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(SFmap)


geometry_schools = gpd.points_from_xy(SFschools['lon'], SFschools['lat'])
schools = gpd.GeoDataFrame(SFschools, geometry=geometry_schools)


for polygon in geojson_data.geometry:

    filtered_schools = schools[schools.geometry.within(polygon)]
    

    for _, row in filtered_schools.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(SFmap)


SFmap


In [96]:
# Load the GeoJSON data
with open("../data/SanFrancisco.Neighborhoods.json") as f:
    geojson_data = gpd.read_file(f)

# Create a GeoDataFrame from the DataFrame with places of interest
geometry = gpd.points_from_xy(SFstarbucks['lon'], SFstarbucks['lat'])
gdf_places = gpd.GeoDataFrame(SFstarbucks, geometry=geometry)

# Create a Folium map
m = folium.Map(location=[gdf_places['lat'].mean(), gdf_places['lon'].mean()], zoom_start=12)

# Iterate through each polygon in the GeoJSON data
for polygon in geojson_data.geometry:
    # Filter the places of interest for each polygon
    filtered_gdf_places = gdf_places[gdf_places.geometry.within(polygon)]
    
    # Add markers for the filtered places of interest
    for _, row in filtered_gdf_places.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(m)


In [93]:
m

In [94]:
with open("../data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson") as f:
    geojson_data = gpd.read_file(f)

# Create a GeoDataFrame from the DataFrame with places of interest
geometry = gpd.points_from_xy(NYCstarbucks['lon'], NYCstarbucks['lat'])
gdf_places = gpd.GeoDataFrame(NYCstarbucks, geometry=geometry)

# Create a Folium map
m = folium.Map(location=[gdf_places['lat'].mean(), gdf_places['lon'].mean()], zoom_start=12)

# Iterate through each polygon in the GeoJSON data
for polygon in geojson_data.geometry:
    # Filter the places of interest for each polygon
    filtered_gdf_places = gdf_places[gdf_places.geometry.within(polygon)]
    
    # Add markers for the filtered places of interest
    for _, row in filtered_gdf_places.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['name']).add_to(m)

m
